# 기본환경 설정

In [ ]:
from google.colab import userdata
HF_KEY = userdata.get("HF_KEY")

In [ ]:
import huggingface_hub
huggingface_hub.login(HF_KEY)

# 모델 로딩

In [1]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 08-06 15:08:02 [__init__.py:235] Automatically detected platform cuda.


In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gemma-3-4b-it",
    load_in_4bit=True
)

==((====))==  Unsloth 2025.7.8: Fast Gemma3 patching. Transformers: 4.54.0. vLLM: 0.10.0.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 2. Max memory: 23.494 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [3]:
model = FastLanguageModel.for_inference(model)

# Prompt 함수

In [4]:
from typing import List, Any, ClassVar
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.outputs import ChatResult, ChatGeneration
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [5]:
class GemmaChatModel(BaseChatModel):
    def __init__(self, model, tokenizer, max_tokens: int = 512):
        super().__init__()
        object.__setattr__(self, "model", model)
        object.__setattr__(self, "tokenizer", tokenizer)
        object.__setattr__(self, "max_tokens", max_tokens)

    @property
    def _llm_type(self) -> str:
        return "gemma-chat"

    def _format_messages(self, messages: List[Any]) -> str:
        prompt = ""
        for message in messages:
            if isinstance(message, SystemMessage):
                prompt += f"<|system|>\n{message.content}</s>\n"
            elif isinstance(message, HumanMessage):
                prompt += f"<|user|>\n{message.content}</s>\n"
            elif isinstance(message, AIMessage):
                prompt += f"<|assistant|>\n{message.content}</s>\n"
        prompt += "<|assistant|>\n"
        return prompt

    def _generate(self, messages: List[Any], **kwargs) -> ChatResult:
        prompt = self._format_messages(messages)
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=self.max_tokens,
                do_sample=True,
                temperature=kwargs.get("temperature", 0.7),
                top_p=kwargs.get("top_p", 0.9),
                eos_token_id=self.tokenizer.eos_token_id,
            )

        decoded = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = decoded.split("<|assistant|>\n")[-1].strip()

        return ChatResult(generations=[ChatGeneration(message=AIMessage(content=response))])

In [6]:
chat_model = GemmaChatModel(model=model, tokenizer=tokenizer, max_tokens=512)

# Prompt 수행

In [7]:
result = chat_model.invoke([
    SystemMessage(content="너는 친절하고 머신러닝 기술을 잘아는 전문가 AI야."),
    HumanMessage(content="LangChain은 무엇인가요?"),
])
print(result.content)

LangChain은 LLM(Large Language Model)을 활용한 애플리케이션 개발을 위한 프레임워크입니다. 간단히 말해, LLM을 더 쉽게 사용할 수 있도록 도와주는 도구 모음이라고 생각하면 됩니다. 

좀 더 자세히 설명하자면, LangChain은 다음과 같은 주요 기능들을 제공합니다.

**1. 핵심 구성 요소:**

*   **모델 (Models):** OpenAI, Google, Hugging Face 등 다양한 LLM과 쉽게 연동할 수 있도록 지원합니다.
*   **프롬프트 (Prompts):** LLM에 제공할 프롬프트를 효과적으로 구성하고 관리하는 데 도움을 줍니다.
*   **체인 (Chains):** 여러 컴포넌트를 연결하여 복잡한 작업을 수행할 수 있도록 합니다. 예를 들어, 프롬프트, 데이터베이스 검색, LLM 호출 등의 단계를 연결하여 하나의 작업 흐름을 만들 수 있습니다.
*   **인덱스 (Indexes):** LLM이 접근할 수 있는 외부 데이터를 저장하고 검색하는 데 사용됩니다. 벡터 데이터베이스, 문서 저장소 등을 활용하여 LLM이 컨텍스트를 이해하고 답변을 생성하는 데 필요한 정보를 제공합니다.
*   **에이전트 (Agents):** LLM이 주어진 작업에 따라 어떤 도구를 사용해야 할지 스스로 결정하고 실행할 수 있도록 합니다. 예를 들어, 검색 엔진, 계산기, 데이터베이스 등 다양한 도구를 활용하여 복잡한 작업을 수행할 수 있습니다.
*   **메모리 (Memory):** LLM이 이전 대화 내용을 기억하고 활용할 수 있도록 합니다. 이를 통해 LLM은 이전 대화의 맥락을 이해하고 일관성 있는 답변을 제공할 수 있습니다.

**2. LangChain의 특징:**

*   **모듈성:** LangChain은 다양한 컴포넌트를 조합하여 원하는 애플리케이션을 구축할 수 있도록 모듈화되어 있습니다.
*   **확장성:** 사용자는 LangChain의 기능을 확장하거나 새로운 컴포넌트를 추가하여 자신만의 애플리케이션을 만

# Chat History In Memory 구성

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [14]:
# Prompt 템플릿 정의 : GemmaChatModel이 원하는 형태로 변경하기 위해.
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])

In [15]:
# 세션별 history 저장 함수
store = {}
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [16]:
# RunnableWithMessageHistory 구성
runnable = RunnableWithMessageHistory(
    prompt | chat_model,  # chat_model은 GemmaChatModel 인스턴스
    get_history,
    input_messages_key="input",
    history_messages_key="history"
)

# session별 Chat History In Memory 수행

# user1 session 테스트

In [17]:
# 세션 아이디 설정
session_id = "user1"

In [18]:
# 4. 대화 실행
response1 = runnable.invoke(
    {"input": "나는 민수라고 해."},
    config={"configurable": {"session_id": session_id}}
)
print("Response 1:", response1.content)

Response 1: 안녕하세요, 민수 씨! 만나서 반갑습니다. 무엇을 도와드릴까요?


In [19]:
response2 = runnable.invoke(
    {"input": "내 이름 기억해?"},
    config={"configurable": {"session_id": session_id}}
)
print("Response 2:", response2.content)

Response 2: 네, 민수 씨라고 말씀하신 것을 기억하고 있습니다. 제가 기억력을 테스트하는 데 도움이 될 만한 질문을 할까요? 아니면 다른 질문이 있으신가요?


In [20]:
# 5. History 출력 (검증)
history = get_history(session_id)
print("\n=== Conversation History ===")
for msg in history.messages:
    print(f"{msg.type.upper()}: {msg.content}")


=== Conversation History ===
HUMAN: 나는 민수라고 해.
AI: 안녕하세요, 민수 씨! 만나서 반갑습니다. 무엇을 도와드릴까요?
HUMAN: 내 이름 기억해?
AI: 네, 민수 씨라고 말씀하신 것을 기억하고 있습니다. 제가 기억력을 테스트하는 데 도움이 될 만한 질문을 할까요? 아니면 다른 질문이 있으신가요?


# user2 session 테스트

In [21]:
# 세션 아이디 설정
session_id = "user2"

In [22]:
response1 = runnable.invoke(
    {"input": "나는 철수라고 해."},
    config={"configurable": {"session_id": session_id}}
)
response2 = runnable.invoke(
    {"input": "내 이름 기억해?"},
    config={"configurable": {"session_id": session_id}}
)
print("Response 2:", response2.content)

Response 2: 네, 철수 씨라고 기억하고 있어요. 혹시 다른 이름으로 불러드릴까요? 아니면 철수 씨에 대해 이야기하고 싶으신가요?


In [23]:
# History 출력 (검증)
history = get_history(session_id)
print("\n=== Conversation History ===")
for msg in history.messages:
    print(f"{msg.type.upper()}: {msg.content}")


=== Conversation History ===
HUMAN: 나는 철수라고 해.
AI: 안녕하세요, 철수 씨! 만나서 반가워요. 무엇을 도와드릴까요?
HUMAN: 내 이름 기억해?
AI: 네, 철수 씨라고 기억하고 있어요. 혹시 다른 이름으로 불러드릴까요? 아니면 철수 씨에 대해 이야기하고 싶으신가요?


# user1 session에 추가 질문

In [24]:
# 세션 아이디 설정
session_id = "user1"

In [25]:
response2 = runnable.invoke(
    {"input": "내 이름 기억해?"},
    config={"configurable": {"session_id": session_id}}
)
print("Response 2:", response2.content)

Response 2: 정직이라는 가치를 중요하게 생각하시는군요. 민수 씨는 성실하고 신뢰할 수


In [26]:
# History 출력 (검증)
history = get_history(session_id)
print("\n=== Conversation History ===")
for msg in history.messages:
    print(f"{msg.type.upper()}: {msg.content}")


=== Conversation History ===
HUMAN: 나는 민수라고 해.
AI: 안녕하세요, 민수 씨! 만나서 반갑습니다. 무엇을 도와드릴까요?
HUMAN: 내 이름 기억해?
AI: 네, 민수 씨라고 말씀하신 것을 기억하고 있습니다. 제가 기억력을 테스트하는 데 도움이 될 만한 질문을 할까요? 아니면 다른 질문이 있으신가요?
HUMAN: 내 이름 기억해?
AI: 정직이라는 가치를 중요하게 생각하시는군요. 민수 씨는 성실하고 신뢰할 수
